In [5]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '용어 식별', '문서 분류']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    labels = {}
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(sorted(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            doc_id = line['documentId']
            if 'claims' not in line:
                labels[doc_id] = line
            else:
                ipc_main = line['ipc_main']
                ipc_all = line['ipc_all'].split('||')
                title = line['invention_title']
                claims = line['claims']
                if 'keyword' in labels[doc_id]:
                    keyword = labels[doc_id]['keyword']
                    keyword = [key.strip() for key in keyword.split('/')]
                else:
                    keyword = None
                #### data preprocess end 

                #### LM(한국어)
                data = {'text': None}
                ## preprocess data from line
                text = f'''<doc>
                    <title>{title}</title>
                    <keyword>{keyword}</keyword>
                    <claims>{claims}</claims>
                </doc>
                '''
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(한국어) end

                if keyword:
                    #### 용어 식별
                    data = {'input': {}, 'output': {}}
                    ## preprocess data from line
                    data['input']['title'] = title
                    data['input']['claims'] = claims
                    data['input']['ipc_main'] = ipc_main
                    data['input']['ipc_all'] = ipc_all
                    data['output']['keyword'] = keyword
                    ## preprocess data from line end
                    task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
                    # print(json.dumps(data, indent=4, ensure_ascii=False))
                    #### 용어 식별 end


                #### 문서 분류
                data = {'input': {}, 'output': {}}
                ## preprocess data from line
                data['input']['title'] = title
                data['input']['claims'] = claims
                data['input']['keyword'] = keyword
                data['output']['ipc_main'] = ipc_main
                data['output']['ipc_all'] = ipc_all
                ## preprocess data from line end
                task_files[2].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### 문서 분류 end


            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 2/2 [00:02<00:00,  1.49s/it]
